In [1]:
import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os
import time

In [4]:
import pandas as pd 
print(np.__version__)

1.24.4


## LOAD DATA

In [2]:
def load_Dataset(baseFile, fold_Number): 
    rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
    base_File = f"{baseFile}/u{fold_Number}.base"
    test_File = f"{baseFile}/u{fold_Number}.test"
    base_Data = pd.read_csv(base_File, sep="\t", header=None, names=rnmColData)
    test_Data = pd.read_csv(test_File, sep="\t", header=None, names=rnmColData)
    base_Data = base_Data.drop(columns=["timestamp"])
    test_Data = test_Data.drop(columns=["timestamp"])
    return base_Data, test_Data

In [3]:
call_base = "ml-100k"
basedata, testdata = load_Dataset(call_base, 1)
basedata


,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
79995,943,1067,2
79996,943,1074,4
79997,943,1188,3
79998,943,1228,3


In [4]:
basedata

,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
79995,943,1067,2
79996,943,1074,4
79997,943,1188,3
79998,943,1228,3


In [5]:
testdata

,user_id,item_id,rating
0,1,6,5
1,1,10,3
2,1,12,5
3,1,14,5
4,1,17,3
...,...,...,...
19995,458,648,4
19996,458,1101,4
19997,459,934,3
19998,460,10,3


In [7]:
rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
readData = pd.read_csv("ml-100k/u.data", sep="\t", header=None, names=rnmColData)

In [5]:
def ConvertRatingMatriks(ratingData):
    # membuat container untuk rating matriks dengan ukuran user x item
    matriks_rating = pd.DataFrame(np.zeros((943, 1682)), columns = list(range(1, 1683)), index = list(range(1, 944)))
    # merubah data frame ke dalam bentuk matriks rating pivot
    convertMatriksRating = ratingData.pivot_table(index = 'user_id', columns = 'item_id', values='rating')
    # mengisi matriks rating NaN dengan 0
    convertMatriksRating = convertMatriksRating.fillna(0)
    # updating matriks rating dengan matriks rating yang sudah di pivot
    matriks_rating.update(convertMatriksRating)
    # mengembalikan matriks rating
    return matriks_rating

## CONVERT RATING MATRIKS

## BASE MODEL

In [6]:
rating_Matriks = ConvertRatingMatriks(basedata)
print(rating_Matriks.shape)
rating_Matriks

# dataset 

(943, 1682)


,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
jumlahRating = np.count_nonzero(rating_Matriks)
jumlahRating

80000

### MEAN 

In [7]:
def MeanRating(RatingMatriks, jenis="user-based"):
    if jenis == "user-based":
        # menghitung berapa banyak rating yang ada pada setiap user(baris)
        Id_axis = 1
        # index baris user dari rating matriks
        index = RatingMatriks.index
    elif jenis == "item-based":
        # menghitung berapa banyak rating yang ada pada setiap item(kolom)
        Id_axis = 0
        # index kolom item dari rating matriks
        index = RatingMatriks.columns
    # menghitung mean rating dari setiap user/item
    # jumlah rating yang ada pada setiap user/item yang bukan 0
    Pembilang = RatingMatriks.sum(axis=Id_axis)
    # user 1
    # print(f"Pembilang user: {Pembilang.loc[941]}")
    # type data Pembilang
    # print(f"Pembilang user : {type(Pembilang.loc[941])}")
    # user 1 pada  item 1 yang telah diberikan rating ada pada setiap user/item yang bukan 0
    Penyebut = np.count_nonzero(RatingMatriks, axis=Id_axis)
    # print(f"Penyebut user : {Penyebut[RatingMatriks.index.get_loc(941)]}")
    # cek jika Penyebut == 0, maka mean rating = 0, jika tidak hitung mean rating
    calculateMeanRating = np.where(Penyebut == 0, 0, Pembilang / Penyebut)

    # mengembalikan mean rating dalam bentuk data frame
    MeanRatingDF = pd.DataFrame(calculateMeanRating, index=index, columns=["meanRating"])
    return MeanRatingDF

In [1]:
import pandas as pd
print(pd.__version__)

1.5.3


#### USER

In [8]:
# vektor, karena mean rating hanya satu kolom, ukuran 943 x 1
cal_MeanRatingUser = MeanRating(rating_Matriks, jenis="user-based")
cal_MeanRatingUser

,meanRating
1,3.681481
2,3.800000
3,3.000000
4,4.357143
5,2.956044
...,...
939,4.265306
940,3.457944
941,4.045455
942,4.265823


#### ITEM

In [9]:
cal_MeanRatingItem = MeanRating(rating_Matriks, jenis="item-based")
print(cal_MeanRatingItem.shape)
cal_MeanRatingItem

(1682, 1)


,meanRating
1,3.892950
2,3.180952
3,3.000000
4,3.526316
5,3.304348
...,...
1678,1.000000
1679,3.000000
1680,2.000000
1681,3.000000


### MEAN-CENTERED

In [ ]:
# user based
# print(f'npRatingMatriks : {NpRatingMatriks[RatingMatriks.index.get_loc(941),:]}')
# print(f'npMean : {npMeanRating[RatingMatriks.index.get_loc(941),:]}')
# print(f'MeanCentered : {MeanCentered[RatingMatriks.index.get_loc(941),:]}')

# item based
# print(f'npRatingMatriks : {NpRatingMatriks[:,RatingMatriks.columns.get_loc(1)]}')
# print(f'npMean : {npMeanRating[0,RatingMatriks.columns.get_loc(1)]}')
# print(f'MeanCentered : {MeanCentered[:,RatingMatriks.columns.get_loc(1)]}')

In [10]:
def MeanCenteredRating(RatingMatriks, meanRating, jenis="user-based"):
    # matriks rating jadi numpy array
    NpRatingMatriks = np.array(RatingMatriks)
    # print(f"ratings matriks : {RatingMatriks.shape}, np rating matriks : {NpRatingMatriks.shape}")

    # print(f'npRatingMatriks : {NpRatingMatriks[:,RatingMatriks.columns.get_loc(1)]}')
    if jenis == "user-based":
        # Reshape meanRating menjadi (jumlah_user, 1), kolom
        npMeanRating = np.array(meanRating).reshape(-1, 1)  # (943, 1)
    elif jenis == "item-based":
        # Reshape meanRating menjadi (1, jumlah_item), baris
        npMeanRating = np.array(meanRating).reshape(1, -1)  # (1, 1650)
    # perhitungan rating setiap user dikurangi mean rating user, yang bukan 0
    MeanCentered = np.where(NpRatingMatriks != 0, NpRatingMatriks - npMeanRating, 0)
    # print(f'npRatingMatriks : {NpRatingMatriks[RatingMatriks.index.get_loc(941),:]}')
    # print(f'npMean : {npMeanRating[RatingMatriks.index.get_loc(941),:]}')
    sudahRated = np.count_nonzero(NpRatingMatriks[RatingMatriks.index.get_loc(941),:])
    # print(f'sudahRated : {sudahRated}')
    nonRated = len(NpRatingMatriks[RatingMatriks.index.get_loc(941),:])
    belum_dinilai = RatingMatriks.loc[941][RatingMatriks.loc[941] == 0].index.tolist()

    # print(f'nonRated : {len(belum_dinilai)}')
    # print(f'MeanCentered : {MeanCentered[RatingMatriks.index.get_loc(941),:]}')

    # Mengembalikan dalam bentuk DataFrame
    dfMeanCentered = pd.DataFrame(MeanCentered, index=RatingMatriks.index, columns=RatingMatriks.columns)
    return dfMeanCentered

#### USER

In [11]:
# matriks
cal_MeanCenteredUser = MeanCenteredRating(rating_Matriks, cal_MeanRatingUser, jenis="user-based")
cal_MeanCenteredUser

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.318519,-0.681481,0.318519,-0.681481,-0.681481,0.0,0.318519,-2.681481,1.318519,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,-1.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.734694,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.000000,0.000000,0.000000,-1.457944,0.000000,0.0,0.542056,1.542056,-0.457944,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.954545,0.000000,0.000000,0.000000,0.000000,0.0,-0.045455,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### ITEM

In [11]:
cal_MeanCenteredItem = MeanCenteredRating(rating_Matriks, cal_MeanRatingItem, jenis="item-based")
print(f"mean cen {cal_MeanCenteredItem.shape}")
cal_MeanCenteredItem

mean cen (943, 1682)


,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.10705,-0.180952,1.0,-0.526316,-0.304348,0.0,0.201954,-2.99422,1.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.10705,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,-1.876712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,1.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.00000,0.000000,0.0,-1.526316,0.000000,0.0,0.201954,1.00578,-0.833333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,1.10705,0.000000,0.0,0.000000,0.000000,0.0,0.201954,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### SIMILARITY

In [ ]:
# print(f"item rated 1", len(itemRated1))
# print(f"item rated ", itemRated1)
# itemNotRated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] == 0])
# print(f"item not rated {itemNotRated1}") 

In [20]:
def SimilarityJaccard(RatingMatriks, index1, index2, jenis="user-based"):
    if jenis == "user-based":
        #mengambil seluru rating dari user pada baris index1 dan index2
        #memili nama item yang sudah di rating oleh user >0 
        itemRated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] != 0])
        # print(f"item rated ", itemRated1)
        itemRated2 = set(RatingMatriks.columns[RatingMatriks.iloc[index2, :] != 0])
    elif jenis == "item-based":
        #mengambil seluru rating untuk item pada kolom index1 dan index2
        #memili nama user yang sudah di rating oleh item >0
        itemRated1 = set(RatingMatriks.index[RatingMatriks.iloc[:, index1] != 0])
        itemRated2 = set(RatingMatriks.index[RatingMatriks.iloc[:, index2] != 0])

    # jumlah item(len)
    intersection = len(itemRated1.intersection(itemRated2))
    union = len(itemRated1.union(itemRated2))
    if union != 0:
        rumusJaccardSim = intersection / union
    else:
        rumusJaccardSim = 0
    return rumusJaccardSim

In [38]:
calSimDuaJaccard = SimilarityJaccard(rating_Matriks, 0, 895, jenis="user-based")
calSimDuaJaccard

0.1585081585081585

In [36]:
def SimilarityAllJaccard(RatingMatriks, jenis="user-based"):
    # jika jenis user-based
    if jenis == "user-based" : 
        # mendapatkan jumlah user(baris) dari rating matriks
        jumlahUser = RatingMatriks.shape[0]
        # membuat container untuk matriks similarity dengan ukuran user x user
        matriksSimilarity = np.zeros((jumlahUser, jumlahUser))
        # mengisi diagonal matriks similarity dengan 1
        np.fill_diagonal(matriksSimilarity, 1)
        # i loop setiap user baris atas
        for i in range (jumlahUser):
            # j loop setiap user lain, mulai dari user ke i+1 agar tidak menghitung user yang sama
            for j in range(i+1, jumlahUser):
                # menghitung jaccard antara user i dan user j
                rumusSimJaccardUser = SimilarityJaccard(RatingMatriks, i, j, jenis="user-based")
                # menyimpan hasil sim dari user i dan j
                matriksSimilarity[i][j] = rumusSimJaccardUser
                matriksSimilarity[j][i] = rumusSimJaccardUser
        # mengembalikan hasil similarity dalam bentuk data frame
        hasilSimJaccard = pd.DataFrame(matriksSimilarity, index=RatingMatriks.index, columns=RatingMatriks.index)
    # jika jenis item-based
    elif jenis == "item-based" :
        # mendapatkan jumlah item(kolom) dari rating matriks 
        jumlahItem = RatingMatriks.shape[1]
        # membuat container untuk matriks similarity dengan ukuran item x item
        matriksSimilarity = np.zeros((jumlahItem, jumlahItem))
        # mengisi diagonal matriks similarity dengan 1
        np.fill_diagonal(matriksSimilarity, 1)
        # i loop setiap item
        for i in range (jumlahItem):
            # j loop setiap item lain, mulai dari item ke i+1 agar tidak menghitung item yang sama
            for j in range(i+1, jumlahItem):
                # menghitung jaccard antara item i dan item j
                rumusSimJaccardItem = SimilarityJaccard(RatingMatriks, i, j, jenis="item-based")
                # menyimpan hasil sim dari item i dan j
                matriksSimilarity[i][j] = rumusSimJaccardItem
                matriksSimilarity[j][i] = rumusSimJaccardItem
        # mengembalikan hasil similarity dalam bentuk data frame
        hasilSimJaccard = pd.DataFrame(matriksSimilarity, index=RatingMatriks.columns, columns=RatingMatriks.columns)
    return hasilSimJaccard

In [265]:
cal_SimiAll = SimilarityAllJaccard(rating_Matriks, jenis="user-based")
cal_SimiAll

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.035503,0.018750,0.013605,0.102439,0.155660,0.101266,0.031250,0.013793,0.106280,...,0.136029,0.041916,0.099206,0.060606,0.080000,0.039548,0.115207,0.046667,0.070000,0.130597
2,0.035503,1.000000,0.046154,0.038462,0.007692,0.094891,0.032653,0.014493,0.061224,0.038760,...,0.033816,0.161765,0.144654,0.230769,0.138462,0.098765,0.088889,0.087719,0.062500,0.034826
3,0.018750,0.046154,1.000000,0.076923,0.008475,0.045455,0.008368,0.054545,0.000000,0.025210,...,0.015075,0.015152,0.055901,0.062500,0.038168,0.026667,0.071429,0.041667,0.038835,0.005128
4,0.013605,0.038462,0.076923,1.000000,0.009615,0.000000,0.013393,0.047619,0.000000,0.009346,...,0.005348,0.019231,0.040000,0.058824,0.033898,0.016129,0.043103,0.090909,0.056818,0.011111
5,0.102439,0.007692,0.008475,0.009615,1.000000,0.086486,0.105455,0.052174,0.019802,0.051136,...,0.147186,0.031746,0.040179,0.015504,0.064171,0.029412,0.093923,0.036697,0.075949,0.146018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.039548,0.098765,0.026667,0.016129,0.029412,0.052980,0.031496,0.012821,0.000000,0.014184,...,0.032407,0.257143,0.136905,0.126582,0.226562,1.000000,0.033113,0.092308,0.024000,0.058537
940,0.115207,0.088889,0.071429,0.043103,0.093923,0.160428,0.134752,0.045802,0.025862,0.161850,...,0.190678,0.050360,0.116592,0.097015,0.091371,0.033113,1.000000,0.057377,0.141104,0.113360
941,0.046667,0.087719,0.041667,0.090909,0.036697,0.039370,0.008584,0.083333,0.062500,0.008696,...,0.015544,0.109091,0.108108,0.127273,0.120690,0.092308,0.057377,1.000000,0.041237,0.027027


In [24]:
cal_SimiAllItemBase = SimilarityAllJaccard(rating_Matriks, jenis="item-based")
cal_SimiAllItemBase

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.000000,0.178744,0.150754,0.236607,0.091787,0.022843,0.416838,0.246637,0.302277,0.106796,...,0.002611,0.0,0.000000,0.000000,0.002611,0.0,0.0,0.0,0.002611,0.002611
2,0.178744,1.000000,0.139241,0.301887,0.175676,0.041667,0.187320,0.168067,0.103774,0.072289,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.009524,0.009524
3,0.150754,0.139241,1.000000,0.160377,0.090909,0.043956,0.147147,0.127273,0.150538,0.112782,...,0.000000,0.0,0.000000,0.000000,0.013333,0.0,0.0,0.0,0.000000,0.013333
4,0.236607,0.301887,0.160377,1.000000,0.170732,0.043716,0.257895,0.269373,0.208696,0.104072,...,0.000000,0.0,0.005848,0.005848,0.005848,0.0,0.0,0.0,0.005848,0.005848
5,0.091787,0.175676,0.090909,0.170732,1.000000,0.011364,0.139394,0.100000,0.097561,0.028986,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1681,0.002611,0.009524,0.000000,0.005848,0.000000,0.000000,0.003257,0.005780,0.004065,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.000000


In [25]:
print(cal_SimiAllItemBase.shape)

(1682, 1682)


### TOP-K

In [2]:
import pandas as pd
print(pd.__version__)


1.5.3


In [45]:
import joblib
cal_Jaccad = joblib.load("case/sim/Jac/ub/simJaccardFold1.joblib")
cal_Jaccad

,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
1,1.000000,0.035503,0.018750,0.013605,0.102439,0.155660,0.101266,0.031250,0.013793,0.106280,...,0.136029,0.041916,0.099206,0.060606,0.080000,0.039548,0.115207,0.046667,0.070000,0.130597
2,0.035503,1.000000,0.046154,0.038462,0.007692,0.094891,0.032653,0.014493,0.061224,0.038760,...,0.033816,0.161765,0.144654,0.230769,0.138462,0.098765,0.088889,0.087719,0.062500,0.034826
3,0.018750,0.046154,1.000000,0.076923,0.008475,0.045455,0.008368,0.054545,0.000000,0.025210,...,0.015075,0.015152,0.055901,0.062500,0.038168,0.026667,0.071429,0.041667,0.038835,0.005128
4,0.013605,0.038462,0.076923,1.000000,0.009615,0.000000,0.013393,0.047619,0.000000,0.009346,...,0.005348,0.019231,0.040000,0.058824,0.033898,0.016129,0.043103,0.090909,0.056818,0.011111
5,0.102439,0.007692,0.008475,0.009615,1.000000,0.086486,0.105455,0.052174,0.019802,0.051136,...,0.147186,0.031746,0.040179,0.015504,0.064171,0.029412,0.093923,0.036697,0.075949,0.146018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.039548,0.098765,0.026667,0.016129,0.029412,0.052980,0.031496,0.012821,0.000000,0.014184,...,0.032407,0.257143,0.136905,0.126582,0.226562,1.000000,0.033113,0.092308,0.024000,0.058537
940,0.115207,0.088889,0.071429,0.043103,0.093923,0.160428,0.134752,0.045802,0.025862,0.161850,...,0.190678,0.050360,0.116592,0.097015,0.091371,0.033113,1.000000,0.057377,0.141104,0.113360
941,0.046667,0.087719,0.041667,0.090909,0.036697,0.039370,0.008584,0.083333,0.062500,0.008696,...,0.015544,0.109091,0.108108,0.127273,0.120690,0.092308,0.057377,1.000000,0.041237,0.027027
942,0.070000,0.062500,0.038835,0.056818,0.075949,0.098837,0.073529,0.028302,0.022472,0.074534,...,0.119469,0.035088,0.057416,0.053097,0.056497,0.024000,0.141104,0.041237,1.000000,0.083333


In [13]:
cal_JaccadItem = joblib.load("case/sim/Jac/ib/simJaccardFold1.joblib")
print(f"sim {cal_JaccadItem.shape}")
cal_JaccadItem

sim (1682, 1682)


,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.000000,0.178744,0.150754,0.236607,0.091787,0.022843,0.416838,0.246637,0.302277,0.106796,...,0.002611,0.0,0.000000,0.000000,0.002611,0.0,0.0,0.0,0.002611,0.002611
2,0.178744,1.000000,0.139241,0.301887,0.175676,0.041667,0.187320,0.168067,0.103774,0.072289,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.009524,0.009524
3,0.150754,0.139241,1.000000,0.160377,0.090909,0.043956,0.147147,0.127273,0.150538,0.112782,...,0.000000,0.0,0.000000,0.000000,0.013333,0.0,0.0,0.0,0.000000,0.013333
4,0.236607,0.301887,0.160377,1.000000,0.170732,0.043716,0.257895,0.269373,0.208696,0.104072,...,0.000000,0.0,0.005848,0.005848,0.005848,0.0,0.0,0.0,0.005848,0.005848
5,0.091787,0.175676,0.090909,0.170732,1.000000,0.011364,0.139394,0.100000,0.097561,0.028986,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1681,0.002611,0.009524,0.000000,0.005848,0.000000,0.000000,0.003257,0.005780,0.004065,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.000000


In [ ]:
# item_not_rated_by = np.where(item_ratings == 0)[0]
# print(f"item not rated by {len(item_not_rated_by)}")

    # print(f"pred unrated {PrediksiUnrated}")
    # Urutkan prediksi menurun dan ambil indeks top-N teratas
    # topNIndex = PrediksiUnrated.argsort()[::-1][:N] #sss, N teratas

## PREDIKSI

In [14]:
import numpy as np
import pandas as pd

def TetanggaK4(target_user, target_item, rating_matrix_np, similarity_np, k, jenis="user-based"):
    """
    Mengembalikan indeks K tetangga terdekat (user atau item) berdasarkan similarity.
    """
    rating_matrix = rating_matrix_np
    if jenis == "user-based":
        # Ambil semua rating terhadap item target
        item_ratings = rating_matrix[:, target_item]
        # User yang pernah memberi rating terhadap item tersebut (kecuali user target)
        tetangga_idx = np.where((item_ratings != 0) & (np.arange(len(item_ratings)) != target_user))[0]
        # Ambil skor similarity
        sim_scores = similarity_np[target_user, tetangga_idx]

    else:  # item-based         
        user_ratings = rating_matrix[target_user, :]
        tetangga_idx = np.where((user_ratings != 0) & (np.arange(len(user_ratings)) != target_item))[0]
        sim_scores = similarity_np[tetangga_idx, target_item]
    # Urutkan tetangga berdasarkan similarity tertinggi
    if len(sim_scores) == 0:
        return []
    # print(f"sim scores {sim_scores}")
    sorted_idx = np.argsort(-sim_scores)[:k]
    # print(f"sorted idx {sorted_idx}")
    # print(f"sim from soreted {sorted_idx} {sim_scores[sorted_idx]}")
    return tetangga_idx[sorted_idx]



In [15]:
def prediksiCF_v2_optimized(RatingMatriks, similarityFunction, mean, meanCen, user, item, k=2, jenis="user-based"):
    """
    Menghitung prediksi rating menggunakan Collaborative Filtering berbasis user/item, versi optimal.
    """

    # Cache ke bentuk NumPy (lebih cepat)
    rating_np = RatingMatriks.values
    similarity_np = similarityFunction.values
    mean_np = mean.values
    meanCen_np = meanCen.values

    # Tetapkan target user dan item sesuai jenis
    target_user, target_item = (user, item) if jenis == "user-based" else (item, user)
    mean_value = mean_np[user if jenis == "user-based" else item]

    # Cari tetangga terdekat
    tetangga = TetanggaK4(target_user, target_item, rating_np, similarity_np, k, jenis)

    if len(tetangga) == 0:
        return float(mean_value)

    # Vektorisasi similarity dan mean centered rating
    if jenis == "user-based":
        sim = similarity_np[user, tetangga]
        mean_cen = meanCen_np[tetangga, item]
    else:
        sim = similarity_np[tetangga, item]
        mean_cen = meanCen_np[user, tetangga]

    # Hitung prediksi
    pembilang = np.dot(mean_cen, sim)
    penyebut = np.sum(np.abs(sim))

    prediksi = mean_value + (pembilang / penyebut) if penyebut != 0 else mean_value
    return float(prediksi)


In [85]:
calOptimisePrediksi = prediksiCF_v2_optimized(rating_Matriks, cal_JaccadItem, cal_MeanRatingItem, cal_MeanCenteredItem, 0, 9, 40, "item-based")
calOptimisePrediksi

3.8805137869975996

In [22]:
def prediksi_semua_matriks(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="user-based"):
    # Salin matriks rating agar tidak mengubah data asli
    hasil_matriks = RatingMatriks.copy()
    num_users, num_items = hasil_matriks.shape
    for user in range(num_users):
        for item in range(num_items):
            if hasil_matriks.iloc[user, item] == 0:
                pred_rating = prediksiCF_v2_optimized(
                    RatingMatriks=RatingMatriks,
                    similarityFunction=similarityFunction,
                    mean=mean,
                    meanCen=meanCen,
                    user=user,
                    item=item,
                    k=k,
                    jenis=jenis
                )
                hasil_matriks.iloc[user, item] = pred_rating
    return hasil_matriks

In [ ]:
calPrediksiSemua = prediksi_semua_matriks(rating_Matriks, cal_Jaccad, cal_MeanRatingUser, cal_MeanCenteredUser, 5, "user-based")
calPrediksiSemua

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.218691,4.000000,1.000000,5.000000,3.728815,...,2.534423,3.544426,3.222151,2.222151,3.469500,1.559986,3.559986,2.559986,3.700818,3.315551
2,4.000000,3.222722,2.963838,3.247616,3.891988,4.830095,3.889122,4.532790,4.440090,2.000000,...,2.652941,3.662944,3.340670,2.340670,3.588018,1.678505,3.678505,2.678505,3.819337,3.434069
3,3.814737,1.785034,2.182020,3.027970,3.085057,2.371788,3.193913,3.120864,3.799709,3.388455,...,1.852941,2.862944,2.540670,1.540670,2.788018,0.878505,2.878505,1.878505,3.019337,2.634069
4,5.046868,3.067127,4.001402,3.844595,3.888865,4.428490,4.923513,4.586607,4.893147,4.506614,...,3.210084,4.220087,3.897813,2.897813,4.145161,2.235648,4.235648,3.235648,4.376480,3.991212
5,3.750135,2.815731,2.322002,2.763512,3.070289,2.139646,3.667401,3.813842,2.751794,3.071424,...,1.808985,2.818988,2.496714,1.496714,2.744062,2.956044,2.956044,2.956044,2.975381,2.590113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.776029,3.958011,4.049760,3.642780,4.181584,5.034501,3.625027,4.829840,5.000000,4.435371,...,3.118247,4.128250,3.805976,2.805976,4.053325,2.143811,4.143811,3.143811,4.284643,3.899376
940,3.403160,3.252267,2.487956,2.000000,2.443996,2.845468,4.000000,5.000000,3.000000,3.779176,...,2.310885,3.320888,2.998614,1.998614,3.245962,1.336449,3.336449,2.336449,3.477281,3.092013
941,5.000000,3.576549,3.975729,2.953050,3.741924,4.123861,4.000000,4.972952,3.971818,3.854685,...,2.898396,3.908399,3.586124,2.586124,3.833473,1.923959,3.923959,2.923959,4.064792,3.679524
942,4.878181,3.814061,3.478054,4.069125,4.053574,4.071380,4.239001,5.162838,4.500917,4.627949,...,3.118764,4.128767,3.806493,2.806493,4.053841,2.144327,4.144327,3.144327,4.285160,3.899892


In [23]:
calPrediksiSemuaItem = prediksi_semua_matriks(rating_Matriks, cal_JaccadItem, cal_MeanRatingItem, cal_MeanCenteredItem, 5, "item-based")
calPrediksiSemuaItem

IndexError: index 943 is out of bounds for axis 0 with size 943

In [28]:
def simpan_semua_prediksi_k(
    RatingMatriks, similarityFunction, mean, meanCen,
    variasi_k, jenis="user-based",
    folder_output="case/prediksi/Jac/ub/5"
):
    for k in variasi_k:
        # Hitung prediksi
        prediksi = prediksi_semua_matriks(
            RatingMatriks=RatingMatriks,
            similarityFunction=similarityFunction,
            mean=mean,
            meanCen=meanCen,
            k=k,
            jenis=jenis
        )

        # Siapkan path file
        path_file = os.path.join(folder_output, f"{k}.joblib")
        os.makedirs(folder_output, exist_ok=True)

        # Simpan hasil prediksi
        joblib.dump(prediksi, path_file)
        print(f"✔️ Prediksi untuk k={k} disimpan di: {path_file}")

In [ ]:
simpan_semua_prediksi_k(
    RatingMatriks=rating_Matriks,
    similarityFunction=cal_Jaccad,
    mean=cal_MeanRatingUser,
    meanCen=cal_MeanCenteredUser,
    variasi_k=[5, 10, 15, 18, 20, 25, 30, 40, 50, 100, 200],
    jenis="user-based",
    folder_output="case/prediksi/Jac/ub/1"
)

NameError: name 'simpan_semua_prediksi_k' is not defined

In [16]:
data = joblib.load("case/prediksi/Jac/ub/1/5.joblib")
print(data.shape)

(943, 1650)


### VARIASI JACCARD UNTUK USER

In [ ]:
variasiParameterKUser = [5, 10, 15, 18, 20, 25, 30, 40, 50, 100, 200]
variasiParameterKItem2= [20]
variasiParameterHybrid = [0.7]

### VARIASI JACCARD UNTUK ITEM

In [ ]:
variasiParameterKUser = [30]
variasiParameterKItem2= [5, 10, 15, 18, 20, 25, 30, 40, 50, 100, 200]
variasiParameterHybrid = [0.7]

### VARIASI JACCARD UNTUK HYBRID

In [ ]:
variasiParameterKUser = [30]
variasiParameterKItem2 = [30]
variasiParameterHybrid = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

## HYBRID

In [94]:
K5UserBased = joblib.load("case/prediksiModel/Jac/ub/1/5.joblib")
K5UserBased

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.218691,4.000000,1.000000,5.000000,3.728815,...,2.534423,3.544426,3.222151,2.222151,3.469500,1.559986,3.559986,2.559986,3.700818,3.315551
2,4.000000,3.222722,2.963838,3.247616,3.891988,4.830095,3.889122,4.532790,4.440090,2.000000,...,2.652941,3.662944,3.340670,2.340670,3.588018,1.678505,3.678505,2.678505,3.819337,3.434069
3,3.814737,1.785034,2.182020,3.027970,3.085057,2.371788,3.193913,3.120864,3.799709,3.388455,...,1.852941,2.862944,2.540670,1.540670,2.788018,0.878505,2.878505,1.878505,3.019337,2.634069
4,5.046868,3.067127,4.001402,3.844595,3.888865,4.428490,4.923513,4.586607,4.893147,4.506614,...,3.210084,4.220087,3.897813,2.897813,4.145161,2.235648,4.235648,3.235648,4.376480,3.991212
5,3.750135,2.815731,2.322002,2.763512,3.070289,2.139646,3.667401,3.813842,2.751794,3.071424,...,1.808985,2.818988,2.496714,1.496714,2.744062,2.956044,2.956044,2.956044,2.975381,2.590113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.776029,3.958011,4.049760,3.642780,4.181584,5.034501,3.625027,4.829840,5.000000,4.435371,...,3.118247,4.128250,3.805976,2.805976,4.053325,2.143811,4.143811,3.143811,4.284643,3.899376
940,3.403160,3.252267,2.487956,2.000000,2.443996,2.845468,4.000000,5.000000,3.000000,3.779176,...,2.310885,3.320888,2.998614,1.998614,3.245962,1.336449,3.336449,2.336449,3.477281,3.092013
941,5.000000,3.576549,3.975729,2.953050,3.741924,4.123861,4.000000,4.972952,3.971818,3.854685,...,2.898396,3.908399,3.586124,2.586124,3.833473,1.923959,3.923959,2.923959,4.064792,3.679524
942,4.878181,3.814061,3.478054,4.069125,4.053574,4.071380,4.239001,5.162838,4.500917,4.627949,...,3.118764,4.128767,3.806493,2.806493,4.053841,2.144327,4.144327,3.144327,4.285160,3.899892


In [95]:
K20ItemBased = joblib.load("case/prediksi/Jac/ib/1/20.joblib")
K20ItemBased

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.734237,4.000000,1.000000,5.000000,4.362603,...,2.900724,4.374306,2.707716,1.707716,3.498950,1.647893,3.647893,2.647893,3.277373,3.172502
2,4.000000,3.384031,3.141674,3.782772,3.509136,3.601089,4.026222,4.272257,4.117270,2.000000,...,3.072647,4.568815,3.133216,2.133216,3.219813,1.136660,3.136660,2.136660,3.265768,3.500937
3,3.314828,2.730854,2.453912,2.889695,2.900274,3.056189,3.194754,3.454956,3.192277,3.354698,...,1.454122,3.427865,2.489079,1.489079,2.836466,0.995517,2.995517,1.995517,3.070928,1.869690
4,4.431340,3.633199,3.641013,3.982584,3.928643,4.498458,4.365417,4.513307,4.478652,4.560093,...,2.066405,4.097348,3.907051,2.907051,3.558575,2.649860,4.649860,3.649860,3.354711,3.122470
5,3.951217,3.268206,2.962717,3.251767,2.796162,2.577517,3.805576,3.719987,3.689707,3.719080,...,1.890317,3.934712,2.496853,1.496853,2.634311,1.000000,3.000000,2.000000,2.490856,2.661462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.983637,4.315029,4.255486,4.702212,4.528999,4.377967,4.884502,5.137675,5.000000,4.995943,...,4.056069,5.102307,3.893030,2.893030,4.201748,2.266180,4.266180,3.266180,4.163715,3.864540
940,3.768573,2.821651,2.827741,2.000000,3.091040,2.859981,4.000000,5.000000,3.000000,3.136807,...,2.339436,3.944146,2.195973,1.195973,2.800176,0.707906,2.707906,1.707906,2.754206,1.994879
941,5.000000,3.557531,3.373307,3.912848,3.688910,3.836516,4.000000,4.424705,4.210614,4.276251,...,3.503121,4.582749,3.483381,2.483381,3.397343,1.449372,3.449372,2.449372,3.238356,3.430639


In [31]:
PrediksiUser = prediksiCF_v2(rating_Matriks, cal_Jaccad, cal_MeanRatingUser, cal_MeanCenteredUser, user=20, item=20, k=10, jenis="user-based")
PrediksiItem = prediksiCF_v2(rating_Matriks, cal_JaccadItem, cal_MeanRatingItem, cal_MeanCenteredItem, user=20, item=20, k=10, jenis="item-based")


sorted idx [52 66 47 31 40 42 33 59 10 48]
top neighbors [775, 918, 698, 485, 633, 647, 533, 842, 255, 707]
sorted idx [46 41 14  2 24  4  5  0  8 12]
top neighbors [558, 442, 239, 6, 280, 14, 16, 0, 122, 199]


In [90]:
def HybridFiltering(PrediksiUser, PrediksiItem, gamma=0.7):
    prediksiUser = np.array(PrediksiUser)
    prediksiItem = np.array(PrediksiItem)
    # Menghitung prediksi hybrid
    rumusHybrid = (gamma * prediksiUser) + ((1 - gamma) * prediksiItem)
    # mengembalikan hasil prediksi hybrid
    return rumusHybrid

In [91]:
calHybridPerUI = HybridFiltering(PrediksiUser, PrediksiItem, gamma=0.7)
calHybridPerUI

NameError: name 'PrediksiUser' is not defined

In [96]:
cal_HybridJ = HybridFiltering(K5UserBased, K20ItemBased, gamma=0.7)

ValueError: operands could not be broadcast together with shapes (943,1682) (943,1650) 

In [97]:
calPrediksi = joblib.load("case/prediksiModel/Jac/ub/3/5.joblib")
calPrediksi

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.000000,3.000000,4.000000,3.000000,3.408279,5.000000,4.000000,1.000000,5.000000,3.000000,...,2.458828,3.467763,3.150237,2.150237,3.360107,1.468311,3.468311,2.468311,3.594444,3.209177
2,4.000000,3.182801,2.451974,2.789982,3.492596,3.861644,3.320686,3.587001,3.712977,2.000000,...,2.479875,3.488809,3.171283,2.171283,3.381154,1.489358,3.489358,2.489358,3.615491,3.230223
3,2.990583,2.320804,2.666947,3.122704,2.571798,2.482461,3.006098,2.506055,3.135567,2.697339,...,1.756061,2.764996,2.447470,1.447470,2.657340,0.765544,2.765544,1.765544,2.891677,2.506410
4,4.689971,3.872394,3.899347,3.712123,4.126628,4.506668,4.823846,4.500423,4.821881,4.000232,...,3.201903,4.210837,3.893311,2.893311,4.103182,2.211386,4.211386,3.211386,4.337519,3.952251
5,4.000000,3.000000,2.170839,3.565921,3.148379,2.487271,3.859619,3.120227,3.416201,3.790595,...,1.746466,2.755400,2.437875,1.437875,2.647745,0.755949,2.755949,1.755949,2.882082,2.496814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.606609,4.681500,3.879048,3.413095,4.280877,4.965565,4.237329,4.404360,5.000000,4.793563,...,3.149027,4.157961,3.840436,2.840436,4.050306,2.158510,4.158510,3.158510,4.284643,3.899376
940,2.917198,2.923143,2.108456,2.000000,3.389555,3.765471,4.000000,5.000000,3.000000,4.181106,...,2.341665,3.350599,3.033073,2.033073,3.242944,1.351148,3.351148,2.351148,3.477281,3.092013
941,5.000000,3.685446,3.054253,3.591836,3.385364,4.187994,4.000000,4.554028,4.365268,3.778381,...,2.929175,3.938110,3.620584,2.620584,3.830455,1.938658,3.938658,2.938658,4.064792,3.679524
942,4.427793,3.611697,3.662604,4.125551,4.117585,4.288883,4.400329,4.635417,5.011295,4.806409,...,3.149544,4.158478,3.840952,2.840952,4.050823,2.159027,4.159027,3.159027,4.285160,3.899892


In [43]:
cal_HybridJ

array([[5.        , 3.        , 4.        , ..., 2.58635808, 3.57378475,
        3.27263618],
       [4.        , 3.27111441, 3.01718858, ..., 2.51595139, 3.65326621,
        3.45412964],
       [3.66476452, 2.06878026, 2.2635876 , ..., 1.91360826, 3.03481431,
        2.40475546],
       ...,
       [5.        , 3.5708434 , 3.79500263, ..., 2.78158291, 3.81686091,
        3.60485851],
       [4.74839323, 3.8248762 , 3.55495679, ..., 2.89663554, 4.06628182,
        3.83048033],
       [4.00159366, 5.        , 4.05478123, ..., 2.9875    , 3.32706309,
        3.03068121]])

In [109]:
# output df 
cal_HybridJ = pd.DataFrame(cal_HybridJ, columns=rating_Matriks.columns, index=rating_Matriks.index)
cal_HybridJ

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.373355,4.000000,1.000000,5.000000,3.918952,...,2.644313,3.793390,3.067821,2.067821,3.478335,1.586358,3.586358,2.586358,3.573785,3.272636
2,4.000000,3.271114,3.017189,3.408163,3.777133,4.461393,3.930252,4.454630,4.343244,2.000000,...,2.778853,3.934706,3.278434,2.278434,3.477557,1.515951,3.515951,2.515951,3.653266,3.454130
3,3.664765,2.068780,2.263588,2.986487,3.029622,2.577108,3.194165,3.221092,3.617479,3.378328,...,1.733295,3.032420,2.525192,1.525192,2.802553,0.913608,2.913608,1.913608,3.034814,2.404755
4,4.862209,3.236948,3.893285,3.885992,3.900798,4.449481,4.756084,4.564617,4.768799,4.522658,...,2.866980,4.183265,3.900584,2.900584,3.969185,2.359911,4.359911,3.359911,4.069949,3.730590
5,3.810460,2.951473,2.514216,2.909989,2.988051,2.271007,3.708853,3.785686,3.033168,3.265721,...,1.833385,3.153705,2.496756,1.496756,2.711137,2.369231,2.969231,2.669231,2.830023,2.611518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.838311,4.065116,4.111478,3.960610,4.285808,4.837541,4.002870,4.922190,5.000000,4.603543,...,3.399594,4.420467,3.832092,2.832092,4.097852,2.180522,4.180522,3.180522,4.248365,3.888925
940,3.512784,3.123083,2.589892,2.000000,2.638109,2.849822,4.000000,5.000000,3.000000,3.586465,...,2.319451,3.507866,2.757822,1.757822,3.112226,1.147886,3.147886,2.147886,3.260358,2.762873
941,5.000000,3.570843,3.795003,3.240989,3.726020,4.037658,4.000000,4.808478,4.043456,3.981155,...,3.079813,4.110704,3.555301,2.555301,3.702634,1.781583,3.781583,2.781583,3.816861,3.604859


In [162]:
cal_HybridJ.iloc[1, 643]


4.235177369206914

## TOP-N REKOMENDASI

In [ ]:
#  print(f"topN item indices {topNItemIndex}")
#     print(f"item indices {topNItemIndex} :")
#     # topNPrediksi = PrediksiRating[topNIndex]
#     # print(f"topN prediksi {topNPrediksi}")
# print(f"topN local idx1 {(topNIndex)}")

In [ ]:
def rekomendasiTopN2(prediksi_array, matriks_rating, user_target, N=5):
    # Ambil rating awal dan prediksi untuk user target
    RatingMatriks = matriks_rating[user_target]
    # print(f"rating row {RatingMatriks}")
    # Ambil prediksi untuk user target, row/baris
    PrediksiRating = prediksi_array[user_target]
    # print(f"pred row {PrediksiRating}")
    # Ambil indeks item yang belum dirating (rating == 0)
    UnratedIndex = np.where(RatingMatriks == 0)[0]
    # print(f"unrated indices {UnratedIndex}")
    # Ambil prediksi hanya untuk item yang belum dirating
    PrediksiUnrated = PrediksiRating[UnratedIndex]
    # print(f"pred unrated {PrediksiUnrated}")
    # Urutkan prediksi menurun dan ambil indeks top-N teratas
    topNIndex = PrediksiUnrated.argsort()[::-1][:N] #sss, N teratas
    # print(f"topN local idx1 {(topNIndex)}")
    # Ambil item-id asli (dari `UnratedIndex`)
    topNItemIndex = UnratedIndex[topNIndex]
    # print(f"topN item indices {topNItemIndex}")
    # print(f"item indices {topNItemIndex} : {PrediksiRating[topNItemIndex]}")
    return topNItemIndex


In [196]:
calTopN = rekomendasiTopN2(cal_HybridJ, rating_Matriks, user_target=1, N=5)
calTopN

topN local idx1 user_id
942    402
941    515
939    490
938    526
936    153
Name: 1, dtype: int64


array([628, 849, 810, 862, 210], dtype=int64)

In [45]:
import joblib
import os

def process_and_save_mean_ratings(baseFile, saveDirectory, jenis="user-based"):
    # Loop untuk 5 fold
    for fold_number in range(1, 6):
        # Load dataset untuk base dan test
        base_Data, _ = load_Dataset(baseFile, fold_number)
        
        # Convert ke rating matriks
        matriks_rating = ConvertRatingMatriks(base_Data)

        simJaccard = SimilarityAllJaccard(matriks_rating, jenis=jenis)
        fileJaccard = f"{saveDirectory}/simJaccardFold{fold_number}.joblib"
        joblib.dump(simJaccard, fileJaccard)
        # joblib



# Contoh penggunaan
baseFile = "ml-100k"  # Ganti dengan path file data Anda
saveSimJUB = "case/sim/Jac/ub"
saveSimJIB = "case/sim/Jac/ib"

process_and_save_mean_ratings(baseFile, saveSimJUB, jenis="user-based")

# # # Menyimpan mean rating untuk item-based
process_and_save_mean_ratings(baseFile, saveSimJIB, jenis="item-based")






In [89]:
dataopen = joblib.load("case/prediksiModel/Jac/ub/2/5.joblib")
print(dataopen.shape)

(943, 1682)
